## A. IMPLEMENTING THE MINSEARCH ENGINE

In [1]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


In [2]:
# minsearch repository https://github.com/alexeygrigorev/minsearch/tree/main

In [3]:
#implementing the search engine 
import minsearch

In [4]:
# Getting the indexed FAQ documents

import json

In [5]:
with open('isw_faq_document.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
# restructuring the json to contain document records for each course
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
#indexing the document so we can search through it 

index = minsearch.Index(
    text_fields = ['question','text'],
    keyword_fields=['course']
)

In [8]:
index.fit(documents)

In [9]:
# texting the index 
#boost = {'question': 2.0, 'section':0.5} #this means the question field is 3 times more important than the text field
boost = {'question': 2.0}

results = index.search(
    query='USSD code for quicketeller recharge?',
    boost_dict =boost,
    filter_dict = {'course':'isw_faq_cleaned'},
    num_results=1
)

In [10]:
results

[{'text': 'Recharge\n*723*[Amount]#\n*723*[MobileNumber]*[Amount]#\nThis is a short code for a self-recharge transaction.\nUse this code to recharge other people’s phone number.\nBalance enquiry\n*723*00#\nYou can use this code to check your account balance. This includes eCash account balance and your other bank accounts. Please note, extra cost may apply and your card must have been added on Quickteller.\nTransfer\n*723*[AccountNumber]*[Amount]#\nUse this code to transfer funds to a bank account.\nGenerate OTP\n*723*0#\nUse this code to generate OTP to complete your transaction.\nOpt out of OTP\n*723*0000#\nUse this code to opt out of using Quickteller USSD service.\nBill Payment\n*723*[PaymentCode]*[CustomerId]*[Amount]#\n*723*[PaymentCode]*[Amount]#\nUse this code to pay bills on Quickteller USSD service.\nLoans\n*723*6#\nAccess Quickteller loans from USSD service.\nPin Selection\n*723*3#\n*723*4#\nUse this code to select a new PIN when you have not activated your eCash.\nUse this 

## B. PASSING THE CONTEXT TO AN LLM 

In [11]:
import os 
#os.environ['OPENAI_API_KEY']

In [12]:
from openai import OpenAI

In [13]:
client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])

def no_rag(query): 
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role":'user', "content":f'{query}'}])
    
    return response.choices[0].message.content

In [14]:
#Setting the context

prompt_template = """
You are course teaching assistant. Answer the QUESTION based on context from the FAQ databse.
Use only the facts from the CONTEXT when answering the QUESTION 
If the CONTEXT doesn't contain the answer , output NONE


QUESTION: {question}

CONTEXT: {context}

"""

In [15]:
context = ""

for doc in results:
    context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"

In [16]:
print(context)

question: What are the USSD codes if I want to do transactions by USSD?
answer: Recharge
*723*[Amount]#
*723*[MobileNumber]*[Amount]#
This is a short code for a self-recharge transaction.
Use this code to recharge other people’s phone number.
Balance enquiry
*723*00#
You can use this code to check your account balance. This includes eCash account balance and your other bank accounts. Please note, extra cost may apply and your card must have been added on Quickteller.
Transfer
*723*[AccountNumber]*[Amount]#
Use this code to transfer funds to a bank account.
Generate OTP
*723*0#
Use this code to generate OTP to complete your transaction.
Opt out of OTP
*723*0000#
Use this code to opt out of using Quickteller USSD service.
Bill Payment
*723*[PaymentCode]*[CustomerId]*[Amount]#
*723*[PaymentCode]*[Amount]#
Use this code to pay bills on Quickteller USSD service.
Loans
*723*6#
Access Quickteller loans from USSD service.
Pin Selection
*723*3#
*723*4#
Use this code to select a new PIN when you

### Modularizing the code 

In [18]:
def search(query):

        # texting the index 
    boost = {'question': 2.0} #this means the question field is 3 times more important than the text field
    
    
    results = index.search(
        query=query,
        boost_dict =boost,
        filter_dict = {'course':'isw_faq_cleaned'},
        num_results=5)

    return results
    

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
    You are course teaching assistant. Answer the QUESTION based on context from the FAQ databse.
    Use only the facts from the CONTEXT when answering the QUESTION 
    If the CONTEXT doesn't contain the answer , output NONE
    
    
    QUESTION: {question}
    
    CONTEXT: {context}
    
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"


    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt



In [20]:
# modularizing the logic for invoking the gpt 

def llm(prompt):
    client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])
    response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role":'user', "content":prompt}])

    return response.choices[0].message.content

In [50]:
# modularized calls 
query = "Can I get a loan extension on Quickteller?"

def rag(query):
 
    search_results = search(query)
    
    prompt = build_prompt(query, search_results)
    
    answer = llm(prompt)

    return answer

In [51]:
no_rag(query)

"As of my last update, Quickteller primarily serves as a payment platform and does not offer direct loan services. However, if you've taken a loan through a partner service that uses Quickteller for payments, you may need to check with that specific service for loan extension options. \n\nTo find out the most accurate and updated information, I recommend visiting the Quickteller website or contacting their customer support directly for assistance regarding loan extensions or any related inquiries."

In [52]:
rag(query)

"To request a loan extension on Quickteller, you can simply dial *561# from your registered number, select 'loans', and then select 'extend Loan'."

## D. PERSISTING DOCUMENT INDEXES USING ELASTICSEARCH FROM A DOCKER CONTAINER

In [24]:
# sample document index from minsearch
documents[15]

{'text': 'Kindly follow the steps below to get your card registered for the OTP service.\nVisit an ATM\nInsert your card\nSelect Quickteller\nSelect "Pay bills"\nChoose your account type\nSelect “Others”\nEnter 322222 as the payment code\nEnter your phone number as “Customer Reference”\nAccept the N1.00 amount displayed\nFollow prompts to complete the activation',
 'question': 'How do I register for a safetoken/OTP?',
 'course': 'isw_faq_cleaned'}

In [25]:
#connecting to the Elastic Search client
#Elastic search saves all the index data on disk - may need to do volume mapping for more 
#advanced persisting.
from elasticsearch import Elasticsearch

In [26]:
es_client  = Elasticsearch('http://localhost:9200')

In [27]:
es_client.info()

ObjectApiResponse({'name': 'ab21dd6fc9ff', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ld5Tw5SlRYORfhg0szWGOQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [28]:
#Creating an index in the database
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "isw-faq"

es_client.indices.create(index=index_name , body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'isw-faq'})

In [41]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
#indexing the FAQ documents - a document is a collection of (question- answer - (document-id or course ))
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 37.10it/s]


In [59]:
#querying the indexed documents 
# "question^3" - means that question is 3 times more important than the text and section fields
# size means we get five results back 

def elastic_search(query) : 
    
    #query = "Where can I find my OTP?"
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^2", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "isw_faq_cleaned"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    
    #collecting the several documents into one List (Constnat time complexity)
    result_docs = [] 
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs




In [60]:
#elastic_search(query)

In [63]:
#Adjusting the RAG workflow to use the elastic search index mappings

# modularized calls 
query = "Can I get a loan extension on Quickteller?"

def rag(query):
 
    search_results = elastic_search(query)
    
    prompt = build_prompt(query, search_results)
    
    answer = llm(prompt)

    return answer



In [65]:
rag(query)

"To request a loan extension on Quickteller, you can simply dial *561# from your registered number, select 'loans', and then select 'extend Loan'."

In [66]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00


## E. Calaculating prompting costs 

In [67]:
import tiktoken

In [68]:
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

In [69]:
search_results = elastic_search(query)

prompt = build_prompt(query, search_results)

len(encoding.encode(prompt))

1030

In [70]:
#gpt mini costs for sychronous API
input_cost = (0.150/1000000) * len(encoding.encode(prompt))
output_cost = (0.6/1000000 ) * len(encoding.encode(rag(query)))

In [83]:
total_cost = "{:.8f}".format(input_cost + output_cost)

naira_cost = "{:.2f}".format(1660*float(total_cost))

In [84]:
tokens = encoding.encode(prompt)[:10]


tokens , print(f"The cost of this prompt is ${total_cost} or #{naira_cost}".format('a'))

The cost of this prompt is $0.00017370 or #0.29


([3575, 553, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122], None)